In [1]:
%matplotlib inline
import pymysql
import os
import re
import copy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from lxml import etree
import json
import time
import numpy as np

In [2]:
connection = pymysql.connect(host='127.0.0.1',
                             user='data',
                             password='data',
                             db='mag-new-160205',
                             charset='utf8mb4',
                             port=3306,
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

In [3]:
def load_result(cursor, container, name):
    for row in cursor.fetchall():
        container.append(row[name])
    return container

In [4]:
conference = 'AAAI'
cursor.execute("SELECT ConferenceSeriesID FROM ConferenceSeries WHERE ShortName = '%s'" % conference)
conferenceID = cursor.fetchall()[0]['ConferenceSeriesID']

In [5]:
cursor.execute("SELECT PaperID, PaperPublishYear FROM Papers WHERE ConferenceSeriesIDMappedToVenueName = '%s'" % conferenceID)
pid_ptime_pairs = []
for row in cursor.fetchall():
    pid_ptime_pairs.append((row['PaperID'], row['PaperPublishYear']))
print (len(pid_ptime_pairs))

12131


In [7]:
co_au = {}
co_count = 0
st_coau = time.time()
for pid, ptime in pid_ptime_pairs:
    cursor.execute("SELECT AuthorID FROM PaperAuthorAffiliations WHERE PaperID = '%s'" % pid)
    aus = []
    load_result(cursor, aus, 'AuthorID')
    for au in aus:
        if au not in co_au.keys():
            co_au[au] = {}
        for cau in aus:
            if au  == cau:
                continue
            if cau not in co_au[au].keys():
                co_au[au][cau] = []
            co_au[au][cau].append(ptime)
            co_count += 1
print ("Searching Time: %.3f, Co-author item count: %d" % (time.time() - st_coau, co_count))


Searching Time: 8.331, Co-author item count: 88516


In [11]:
def export_co_author(co_dict, filename):
    f = open(filename, 'w')
    for au in co_dict.keys():
        if not co_dict[au]:
            f.write(au+'\n')
            continue
        f.write(au+'\t')
        for cau in co_dict[au].keys():
            f.write(cau+'\t')
            for i in range(len(co_dict[au][cau])-1):
                f.write(str(co_dict[au][cau][i])+ ' ' + str(co_dict[au][cau][i])+'\t')
            f.write(str(co_dict[au][cau][-1]) + ' ' + str(co_dict[au][cau][-1])+'\n')
    f.close()
    print ("export %s." % filename)

In [12]:
export_co_author(co_au, 'data/coau/link_%s.txt' % conference)

export data/coau/link_AAAI.txt.
